In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from nltk.corpus import stopwords
import regex as re

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
from sklearn.svm import SVC
#Forests

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning);

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score

from random import choices

import pickle

In [5]:
with open('model_base.pkl', 'rb') as f: #reading bytes
    foo = pickle.load(f)

In [ ]:
df = pd.read_csv('../data/cleaned_02_ii.csv')
print(df.shape)
df.head()

In [6]:
foo.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('tvec',
                 TfidfVectorizer(lowercase=False, max_df=0.95, min_df=0.0,
                                 ngram_range=(1, 2))),
                ('lr',
                 LogisticRegression(class_weight={0: 0.8, 1: 3, 2: 6},
                                    max_iter=3000,
                                    multi_class='multinomial'))])>

In [ ]:
lower_limit, upper_limit = .5, 2
df['hate_rank'] = [2 if i > upper_limit
                  else 1 if i > lower_limit else 0
                  for i in df['hate_speech_score']]
df['hate_rank'].value_counts(normalize=True)

In [ ]:
X = df['cleaned_text']
y = df['hate_rank']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 3785, #To have a clean 200 2s in the test 3785 pref
                                                    random_state = 26, #I like this number
                                                    stratify=y) #Particularly important in this case with 3 targets

# Hmm, as it stands right now y_test has mostly 0s... eh, I think htat's fine as it's like new data. So, cvs might be a little
# scruff now with seeing thet sam thing,b ut eh, better htan nothing.... And, likely such is the inhernt issue of
# using such methodologies...

In [ ]:
data = pd.DataFrame(X_train)
data['hate_rank'] = y_train
data.head()

In [ ]:
data_names = []
for i in range(3):
    data_names.append(f"data_{i}")
data_names

In [ ]:
for i in range(len(data_names)):#Recall that we're still planning on doing replace=True even for 0.
    data_names[i] = data[data['hate_rank']==i].sample(col_size, replace=True, random_state=26)

In [ ]:
data = pd.concat(data_names)
print(data.shape)
data.head()
X_train = data['cleaned_text']
y_train = data['hate_rank']

In [ ]:
# cvec_ex = CountVectorizer(stop_words='english')
# X_train_cvec_ex = cvec_ex.fit_transform(X_train)
# X_train_ex = pd.DataFrame(X_train_cvec_ex.todense(),
#                           columns=cvec_ex.get_feature_names_out())

# X_train_ex.sum().sort_values().tail(30).plot(kind="barh");

In [ ]:
# trial_model.named_steps['lr'].coef_[0]

In [ ]:
# trial_model.named_steps['tvec'].get_feature_names_out()

In [ ]:
def custom_recall(y, ypred, weight_list):
    numer = 0
    for i in range(len(weight_list)):
        numer += weight_list[i] * recall_score(y, ypred, average=None)[i]
    return numer/sum(weight_list)

In [ ]:
current_weights={
    0:.8
    ,1:3
    ,2:6
}
cw_values = list(current_weights.values())
print(cw_values)

In [ ]:
# pipe = Pipeline([
#     ('tvec', TfidfVectorizer(lowercase=False))
#     ,('lr', LogisticRegression(max_iter=3000, multi_class='multinomial', class_weight=current_weights))
# ])
# pipe_params = {'tvec__min_df' : [.0]
#                ,'tvec__max_df' : [.95]
#                ,'tvec__ngram_range' : [(1,2)]
#                ,'tvec__max_features' : [None]
#                ,'tvec__stop_words' : [None]
#                }
# scorer = make_scorer(custom_recall, weight_list=cw_values[1:]) #Note the 1: as we're now ignoring 0 for the scoring
# gs = GridSearchCV(pipe
#                   ,param_grid=pipe_params
#                   ,cv=5
#                   ,scoring=scorer
#                  )

# gs.fit(X_train, y_train)

# print(gs.score(X_train, y_train), gs.score(X_test, y_test))
# print(gs.best_score_)
# print(gs.best_params_)

In [ ]:
trial_model = gs.best_estimator_
trial_model.fit(X_train,y_train)
print(trial_model.score(X_train, y_train), trial_model.score(X_test, y_test))

preds = trial_model.predict(X_test)
preds_prob = trial_model.predict_proba(X_test)
hyper_pred_comparer = pd.DataFrame({
    'cleaned_text':X_test
    ,'actual':y_test
    ,'pred':preds
    ,'p_0':preds_prob[:,0]
    ,'p_1':preds_prob[:,1]
    ,'p_2':preds_prob[:,2]
})

ConfusionMatrixDisplay.from_estimator(trial_model,X_test,y_test, cmap='Greens');

In [7]:
##0.6277581120943954
class_twos = confusion_matrix(y_test, preds)[2]
class_ones = confusion_matrix(y_test, preds)[1]
(cw_values[1]*(class_ones[1]/sum(class_ones))+cw_values[2]*(class_twos[2]/sum(class_twos)))/sum(cw_values[1:]) 

NameError: name 'confusion_matrix' is not defined